In [1]:
using CSV
using DataFrames

In [2]:
include("PARAMETERS.jl");

# Read and format the spreadsheet of ion reactions from Roger

In [2]:
pwd()

"/home/emc/GDrive-CU/Research-Modeling/UpperAtmoDH/Code"

In [110]:
ionfile = "../Resources/ireactions-full-copy.csv"

"../Resources/ireactions-full-copy.csv"

In [111]:
function format_ion_network(ion_rxn_file)
    
    # Open the CSV, send it into a data frame, get rid of missing values, and construct an array to store the network
    ion_table = CSV.File(ion_rxn_file) |> DataFrame
    replace!(ion_table."P3", missing=>"-0");
    numrows = size(ion_table)[1]
    ion_network = Array{Any}(undef, numrows, 1)

    # Fill in the network
    for i in range(1, stop=numrows)
    
        # set up the reaction - don't need to worry about termolecular ion rxns
        reactants = [Symbol(ion_table[i, "R1"]), Symbol(ion_table[i, "R2"])]
        products = [ion_table[i, "P1"], ion_table[i, "P2"], ion_table[i, "P3"]]
        products = filter!(j->j!="-0", products)
        products = [Symbol(j) for j in products]

        # electron vs. ion temperature
        Tstr = ion_table[i, :].type == -4 ? "Te" : "Ti"

        # construct the symbolic rate expression
        k_A = :(2 .* $(ion_table[i, "kA"])) #Meta.parse("2 .* $(string(ion_table[i, :].kA))")
        # println(k_A)
        if ion_table[i, :].kB != 0
            k = :($k_A .* $(Meta.parse(Tstr)) .^ $(ion_table[i, "kB"]))

            # k_B = ".*($(Tstr) .^ $(ion_table[i, :].kB))" # old 
        else
            k = k_A
            # k_B = "" # old
        end

        if ion_table[i, :].kC != 0
            k = :($k .* exp.($(ion_table[i, "kC"]) ./ $(Meta.parse(Tstr))))
            # k_C = ".* exp.($(ion_table[i, :].kC) ./ $(Tstr))" # old
        else
            k = k
            # k_C = "" # old
        end

        ion_network[i] = [reactants, products, k]   #:($k_A * $k_B * $k_C)] # old and not perfect  
        end
        
        return ion_network
    
end

format_ion_network (generic function with 1 method)

In [112]:
format_ion_network(ionfile)

582×1 Matrix{Any}:
 Any[[:ArHpl, :C], [:CHpl, :Ar], :(2 .* 1.02e-9)]
 Any[[:ArHpl, :CO], [:HCOpl, :Ar], :(2 .* 1.25e-9)]
 Any[[:ArHpl, :CO2], [:HCO2pl, :Ar], :(2 .* 1.1e-9)]
 Any[[:ArHpl, :H2], [:H3pl, :Ar], :(2 .* 6.3e-10)]
 Any[[:ArHpl, :N2], [:N2Hpl, :Ar], :(2 .* 8.0e-10)]
 Any[[:ArHpl, :O], [:OHpl, :Ar], :(2 .* 5.9e-10)]
 Any[[:ArHpl, :O2], [:HO2pl, :Ar], :(2 .* 5.05e-10)]
 Any[[:Arpl, :CO], [:COpl, :Ar], :(2 .* 4.4e-11)]
 Any[[:Arpl, :CO2], [:CO2pl, :Ar], :(2 .* 4.8e-10)]
 Any[[:Arpl, :H2], [:ArHpl, :H], :(2 .* 8.72e-10)]
 Any[[:Arpl, :H2], [:H2pl, :Ar], :(2 .* 1.78e-11)]
 Any[[:Arpl, :H2O], [:ArHpl, :OH], :(2 .* 3.24e-10)]
 Any[[:Arpl, :H2O], [:H2Opl, :Ar], :(2 .* 1.3e-9)]
 ⋮
 Any[[:NH2pl, :E], [:N, :H, :H], :(((2 .* 1.71e-5) .* Te .^ -0.8) .* exp.(-17.1 ./ Te))]
 Any[[:NH2pl, :E], [:NH, :H], :(((2 .* 8.34e-6) .* Te .^ -0.79) .* exp.(-17.1 ./ Te))]
 Any[[:NH3pl, :E], [:NH, :H, :H], :((2 .* 2.68e-6) .* Te .^ -0.5)]
 Any[[:NH3pl, :E], [:NH2, :H], :((2 .* 2.68e-6) .* Te .^ -0.5)]
 A

# Same but for neutrals

In [8]:
neutralfile = "/home/emc/GDrive-CU/Research-Modeling/UpperAtmoDH/Resources/nreactions_pared.csv"

"/home/emc/GDrive-CU/Research-Modeling/UpperAtmoDH/Resources/nreactions_pared.csv"

## rate coeff funcs

In [100]:
function make_general_k(A, B, C)
    #=
    Construct a symbolic expression of the form A * T^B * exp(C/Tn)
    =#
    k_A = :($A) 
    if B != 0
        k = :($k_A .* (Tn .^ $B))     
    else
        k = k_A
    end
    if C != 0
        k = :($k .* exp.($C ./ Tn))
    else
        k = k
    end

    
    return k
end


function construct_FF(k0, kinf, F)
    #=
    Put together a really nasty expression
    =#
    outer_numer = :(log10.($(F)))
    inner_numer = :(log10.(($(k0) .* M) ./ ($(kinf))) .- 0.4 .- 0.67 .* log10.($(F)))
    inner_denom = :(0.75 .- 1.27 .* log10.($(F)) .- 0.14 .* (log10.(($(k0) .* M) ./ ($(kinf))) .- 0.4 .- 0.67 .* log10.($(F))))
    outer_denom = :(1 .+ (($(inner_numer)) ./ ($(inner_denom))) .^ 2)
    FF = :(10 .^ (($(outer_numer)) ./ ($(outer_denom))))
    return FF
end


function make_Troe(k0_ABC, kinf_ABC, F)
    #=
    k0_ABC and kinf_ABC are lists of A, B, and C for k0 and kinf.
    =#
    k0 = make_general_k(k0_ABC...)
    kinf = make_general_k(kinf_ABC...)
    
    if F == 0
        return :(($(k0) .* $(kinf) .* M) ./ ($(k0) .* M .+ $(kinf)))  # Updated to match Roger's code 4 Feb 2021
    else        
        FF = construct_FF(k0, kinf, F)
        return :($(FF) .* ($(k0) .* $(kinf)) ./ ($(k0) .* M .+ $(kinf)))  # Updated to match Roger's code 5 Feb 2021
    end
end

function make_modified_Troe(k0_ABC, kinf_ABC, kR_ABC, F)
    #=
    "Type 4" pressure dependent association reactions. 
    Roger has the result for F=0 in his code as k_tot = k2 + k0*kinf*M/(k0*M+kinf), with k2 (kR in the appendix from Vuitton)
    being the general form and using the third set of A, B, C coefficients in his file. Those columns are 0 in the file he gave me, but
    I have coded it like this though to make sure to keep the form in case I ever get one where it's not 0.
    
    k0_ABC and kinf_ABC are lists of A, B, and C for k0 and kinf.
    =#
    k0 = make_general_k(k0_ABC...)
    kinf = make_general_k(kinf_ABC...)
    kR = make_general_k(kR_ABC...)
    
    if F == 0
        return :($(kR) .+ ($(k0) .* $(kinf) .* M) ./ ($(k0) .* M .+ $(kinf)))
    else
        FF = construct_FF(k0, kinf, F)
        
        # for this one, we need the minimum of k_inf and a more complicated expression.
        # Format for minimum of A and B is: :(min.($:(A), $:(B)))
        return :(min.($:($kinf), $:($(kR) .+ ($(FF) .* $(k0) .* $(kinf) .* M) ./ ($(k0) .* M .+ $(kinf)))))
    end
end

function all_reactants_used(d)
    #=
    Filter out reactions with reactants we won't use 
    =# 
    
    r1 = Symbol(d["R1"])
    r2 = Symbol(d["R2"])
    
    unused = []

    if !(r1 in all_species)
        # println("$(r1) + $(r2) --> $(d["P1"]) + $(d["P2"]) + $(d["P3"]) contains an unused species as reactant, deleting...")
        push!(unused, r1)
        return false
    else  # if the first reactant is used
        if d["R2"] != "-0"
            if !(r2 in all_species)
                # println("$(r1) + $(r2) --> $(d["P1"]) + $(d["P2"]) + $(d["P3"]) contains unused species $(r2) as reactant, deleting...")
                push!(unused, r2)
                return false
            else
                return true
            end 
        else
            return true
        end
    end

    println("Deleted all reactions containing these species: $(unused)")
end

function format_neutral_network(neutralfile)
    #=
    Construct array of the neutral network.
    
    =#
    
    # Load the neutral network, replace missing values
    ntable = CSV.File(neutralfile) |> DataFrame
    replace!(ntable."P2", missing => "-0");
    replace!(ntable."P3", missing => "-0");
    select!(ntable, Not(:Column1))  # Ignore the index column
    
    ntable = filter(all_reactants_used, ntable)
    
    # Set up the array to store neutral network 
    numrows = size(ntable)[1]
    neutral_network = Array{Any}(undef, numrows, 1)

    # identify the type and sort into lists for some cursory perusal
    type1 = Any[]
    type2 = []
    type3 = []
    type4 = []
    type5 = []
    
    # Fill the network
    
    for i in range(1, stop=numrows)
        # set up the reaction
        reactants = [ntable[i, "R1"], ntable[i, "R2"]]
        reactants = filter!(j->j!="-0", reactants)
        reactants = [Symbol(j) for j in reactants]
        products = [ntable[i, "P1"], ntable[i, "P2"], ntable[i, "P3"]]
        products = filter!(j->j!="-0", products)
        products = [Symbol(j) for j in products]

        # Type 1: Pressure independent unimolecular rxns, high pressure limit
        # use k (which is same as k inf); units s^-1
        if ntable[i, "type"]==1
            push!(type1, [reactants, products, :($(make_general_k(ntable[i, "kA"], ntable[i, "kB"], ntable[i, "kC"])))   ])

        # Type 2: P independent bimolecular; use k (same as *k inf), units cm^3 s^-1
        elseif ntable[i, "type"]==2
            push!(type2, [reactants, products, make_general_k(ntable[i, "kA"], ntable[i, "kB"], ntable[i, "kC"])])

        # Type 3: P dependent bimolecular.
        elseif ntable[i, "type"]==3
            push!(type3, [reactants, products, 
                          make_Troe([ntable[i, "k0A"], ntable[i, "k0B"], ntable[i, "k0C"]], 
                                   [ntable[i, "kA"], ntable[i, "kB"], ntable[i, "kC"]],
                                    ntable[i, "F"])
                          ])

        # Type 4: P dependent association rxns
        elseif ntable[i, "type"]==4
            push!(type4, [reactants, products, 
                          make_modified_Troe([ntable[i, "k0A"], ntable[i, "k0B"], ntable[i, "k0C"]], 
                                             [ntable[i, "kA"], ntable[i, "kB"], ntable[i, "kC"]],
                                             [ntable[i, "kradA"], ntable[i, "kradB"], ntable[i, "kradC"]],
                                             ntable[i, "F"])

                         ])

        # Type 5: special forms for OH+CO -> H + CO2, OH+CO -> HOCO
        elseif ntable[i, "type"]==5
            push!(type5, [reactants, products, make_general_k(ntable[i, "k0A"], ntable[i, "k0B"], ntable[i, "k0C"]), 
                                               make_general_k(ntable[i, "kA"], ntable[i, "kB"], ntable[i, "kC"])])

        else
            println("Got an unexpected type! $(ntable[i, "type"])")
        end
    end
    
    return type1, type2, type3, type4, type5
    
end


format_neutral_network (generic function with 1 method)

## Test the functions that provide the rate coefficients

In [101]:
k0_ABC = [5, -0.5, 0]
kinf_ABC = [5, -0.6, -200]

k0_test = make_general_k(k0_ABC...)
kinf_test = make_general_k(kinf_ABC...)

:((5.0 .* Tn .^ -0.6) .* exp.(-200.0 ./ Tn))

### construct_FF

In [102]:
construct_FF_old(k0_test, kinf_test, 2.2)

"10 .^ ((log10.(2.2)) ./ (1 .+ ((log10.((5.0 .* Tn .^ -0.5 .* M) ./ ((5.0 .* Tn .^ -0.6) .* exp.(-200.0 ./ Tn))) .- 0.4 .- 0.67 .* log10.(2.2)) ./ (0.75 .- 1.27 .* log10.(2.2) .- 0.14 .* (log10.((5.0 .* Tn .^ -0.5 .* M) ./ ((5.0 .* Tn .^ -0.6) .* exp.(-200.0 ./ Tn))) .- 0.4 .- 0.67 .* log10.(2.2)))) .^ 2))"

In [103]:
construct_FF(k0_test, kinf_test, 2.2)

:(10 .^ (log10.(2.2) ./ (1 .+ (((log10.(((5.0 .* Tn .^ -0.5) .* M) ./ ((5.0 .* Tn .^ -0.6) .* exp.(-200.0 ./ Tn))) .- 0.4) .- 0.67 .* log10.(2.2)) ./ ((0.75 .- 1.27 .* log10.(2.2)) .- 0.14 .* ((log10.(((5.0 .* Tn .^ -0.5) .* M) ./ ((5.0 .* Tn .^ -0.6) .* exp.(-200.0 ./ Tn))) .- 0.4) .- 0.67 .* log10.(2.2)))) .^ 2)))

### make_Troe

In [104]:
make_Troe_old(k0_ABC, kinf_ABC, 0)

Symbol("(5.0 .* Tn .^ -0.5 .* (5.0 .* Tn .^ -0.6) .* exp.(-200.0 ./ Tn) .* M) ./ (5.0 .* Tn .^ -0.5 .* M .+ (5.0 .* Tn .^ -0.6) .* exp.(-200.0 ./ Tn))")

In [105]:
make_Troe(k0_ABC, kinf_ABC, 0)

:((((5.0 .* Tn .^ -0.5) .* ((5.0 .* Tn .^ -0.6) .* exp.(-200.0 ./ Tn))) .* M) ./ ((5.0 .* Tn .^ -0.5) .* M .+ (5.0 .* Tn .^ -0.6) .* exp.(-200.0 ./ Tn)))

### make_modified_TRoe

In [106]:
make_modified_Troe_old(k0_ABC, kinf_ABC, [4, -0.9, -600], 0)

Symbol("(4.0 .* Tn .^ -0.9) .* exp.(-600.0 ./ Tn) .+ (5.0 .* Tn .^ -0.5 .* (5.0 .* Tn .^ -0.6) .* exp.(-200.0 ./ Tn) .* M) ./ (5.0 .* Tn .^ -0.5 .* M .+ (5.0 .* Tn .^ -0.6) .* exp.(-200.0 ./ Tn))")

In [107]:
make_modified_Troe(k0_ABC, kinf_ABC, [4, -0.9, -600], 0)

:((4.0 .* Tn .^ -0.9) .* exp.(-600.0 ./ Tn) .+ (((5.0 .* Tn .^ -0.5) .* ((5.0 .* Tn .^ -0.6) .* exp.(-200.0 ./ Tn))) .* M) ./ ((5.0 .* Tn .^ -0.5) .* M .+ (5.0 .* Tn .^ -0.6) .* exp.(-200.0 ./ Tn)))

#### make_modified_Troe when F !=0

In [108]:
# O + N --> NO
make_modified_Troe([5.46e-33, 0, 155], [1, 0, 0], [0,0,0], 0.4)

:(min.(1, 0 .+ (((10 .^ (log10.(0.4) ./ (1 .+ (((log10.(((5.46e-33 .* exp.(155.0 ./ Tn)) .* M) ./ 1) .- 0.4) .- 0.67 .* log10.(0.4)) ./ ((0.75 .- 1.27 .* log10.(0.4)) .- 0.14 .* ((log10.(((5.46e-33 .* exp.(155.0 ./ Tn)) .* M) ./ 1) .- 0.4) .- 0.67 .* log10.(0.4)))) .^ 2)) .* (5.46e-33 .* exp.(155.0 ./ Tn))) .* 1) .* M) ./ ((5.46e-33 .* exp.(155.0 ./ Tn)) .* M .+ 1)))

## Try the neutral network

In [92]:
t1, t2, t3, t4, t5 = format_neutral_network(neutralfile);

In [109]:
t2

183-element Vector{Any}:
 Any[[:C, :C], [:C2], 2.16e-11]
 Any[[:C, :H], [:CH], 1.0e-17]
 Any[[:C, :N], [:CN], :((6.93e-20 .* Tn .^ 0.37) .* exp.(-51.0 ./ Tn))]
 Any[[:CH, :C], [:C2, :H], 6.59e-11]
 Any[[:CH, :H], [:H2, :C], :(1.31e-10 .* exp.(-80.0 ./ Tn))]
 Any[[:CH, :H2], [:CH2, :H], :(2.9e-10 .* exp.(-1670.0 ./ Tn))]
 Any[[:CH, :H2], [:CH3], :((2.92e-16 .* Tn .^ -0.71) .* exp.(-11.6 ./ Tn))]
 Any[[:CH, :N], [:CN, :H], :(2.77e-10 .* Tn .^ -0.09)]
 Any[[:CH, :O], [:CO, :H], 6.6e-11]
 Any[[:CH, :O], [:HCOP, :E], :(4.2e-13 .* exp.(-850.0 ./ Tn))]
 Any[[:CH, :O], [:OH, :C], :(2.52e-11 .* exp.(-2381.0 ./ Tn))]
 Any[[:CN, :H2], [:HCN, :H], :((1.8e-19 .* Tn .^ 2.6) .* exp.(-960.0 ./ Tn))]
 Any[[:CN, :N], [:N2, :C], :(9.8e-10 .* Tn .^ -0.4)]
 ⋮
 Any[[:OH, :C], [:CO, :H], :((7.98e-10 .* Tn .^ -0.34) .* exp.(-0.108 ./ Tn))]
 Any[[:OH, :CH], [:HCO, :H], :((8.31e-13 .* Tn .^ 0.5) .* exp.(-5000.0 ./ Tn))]
 Any[[:OH, :H], [:H2, :O], :((8.1e-21 .* Tn .^ 2.8) .* exp.(-1950.0 ./ Tn))]
 Any[[:OH, :H2]

# older code

In [7]:
neutral_table = CSV.File("/home/emc/GDrive-CU/Research-Modeling/UpperAtmoDH/Resources/nreactions_pared.csv") |> DataFrame
replace!(neutral_table."P2", missing => "-0");
replace!(neutral_table."P3", missing => "-0");
select!(neutral_table, Not(:Column1))

,R1,R2,P1,P2,P3,type,kA,kB,kC,k0A
,String,String,String,String,String?,Int64,Float64,Float64,Float64,Float64
1,C,C,C2,-0,-0,2,2.16e-11,0.0,0.0,0.0
2,C,H,CH,-0,-0,2,1.0e-17,0.0,0.0,0.0
3,C,N,CN,-0,-0,2,6.93e-20,0.37,-51.0,0.0
4,CH,C,C2,H,-0,2,6.59e-11,0.0,0.0,0.0
5,CH,H,H2,C,-0,2,1.31e-10,0.0,-80.0,0.0
6,CH,H2,CH2,H,-0,2,2.9e-10,0.0,-1670.0,0.0
7,CH,H2,CH2,H,-0,4,8.5e-11,0.15,0.0,4.7e-26
8,CH,H2,CH3,-0,-0,2,2.92e-16,-0.71,-11.6,0.0
9,CH,N,CN,H,-0,2,2.77e-10,-0.09,0.0,0.0


In [4]:
numrows = size(neutral_table)[1]
neutral_network = Array{Any}(undef, numrows, 1)

# identify the type and sort into lists for some cursory perusal
type1 = []
type2 = []
type3 = []
type4 = []
type5 = []


make_modified_Troe (generic function with 1 method)

In [5]:
for i in range(1, stop=numrows)
    # set up the reaction
    reactants = [neutral_table[i, "R1"], neutral_table[i, "R2"]]
    reactants = filter!(j->j!="-0", reactants)
    reactants = [Symbol(j) for j in reactants]
    products = [neutral_table[i, "P1"], neutral_table[i, "P2"], neutral_table[i, "P3"]]
    products = filter!(j->j!="-0", products)
    products = [Symbol(j) for j in products]
    
    # Type 1: Pressure independent unimolecular rxns, high pressure limit
    # use k (which is same as k inf); units s^-1
    if neutral_table[i, "type"]==1
        push!(type1, [reactants, products, Symbol(make_general_k(neutral_table[i, "kA"], neutral_table[i, "kB"], neutral_table[i, "kC"]))])
        
    # Type 2: P independent bimolecular; use k (same as *k inf), units cm^3 s^-1
    elseif neutral_table[i, "type"]==2
        push!(type2, [reactants, products, Symbol(make_general_k(neutral_table[i, "kA"], neutral_table[i, "kB"], neutral_table[i, "kC"]))])
        
    # Type 3: P dependent bimolecular.
    elseif neutral_table[i, "type"]==3
        push!(type3, [reactants, products, 
                      make_Troe([neutral_table[i, "k0A"], neutral_table[i, "k0B"], neutral_table[i, "k0C"]], 
                               [neutral_table[i, "kA"], neutral_table[i, "kB"], neutral_table[i, "kC"]],
                                neutral_table[i, "F"])
                      ])
        
    # Type 4: P dependent association rxns
    elseif neutral_table[i, "type"]==4
        push!(type4, [reactants, products, 
                      make_modified_Troe([neutral_table[i, "k0A"], neutral_table[i, "k0B"], neutral_table[i, "k0C"]], 
                                         [neutral_table[i, "kA"], neutral_table[i, "kB"], neutral_table[i, "kC"]],
                                         [neutral_table[i, "kradA"], neutral_table[i, "kradB"], neutral_table[i, "kradC"]],
                                         neutral_table[i, "F"])
                
                     ])
        
    # Type 5: special forms for OH+CO -> H + CO2, OH+CO -> HOCO
    elseif neutral_table[i, "type"]==5
        push!(type5, [reactants, products, Symbol(make_general_k(neutral_table[i, "k0A"], neutral_table[i, "k0B"], neutral_table[i, "k0C"])), 
                                           Symbol(make_general_k(neutral_table[i, "kA"], neutral_table[i, "kB"], neutral_table[i, "kC"]))])

    else
        println("Got an unexpected type! $(neutral_table[i, "type"])")
    end
end


    
println("Type 1:")
for entry in type1
    println(entry)
end
println()
println("Type 2:")
for entry in type2
    println(entry)
end
println()
println("Type 3:")
for entry in type3
    println(entry)
end
println()
println("Type 4:")
for entry in type4
    println(entry)
    println()
end
println("Type 5:")
for entry in type5
    println(entry)
end
# println(neutral_network)


Type 1:
Any[[:O1D], [:O], Symbol("0.0051")]

Type 2:
Any[[:C, :C], [:C2], Symbol("2.16e-11")]
Any[[:C, :H], [:CH], Symbol("1.0e-17")]
Any[[:C, :N], [:CN], Symbol("6.93e-20 .* (Tn .^ 0.37) .* exp.(-51.0 ./ Tn)")]
Any[[:CH, :C], [:C2, :H], Symbol("6.59e-11")]
Any[[:CH, :H], [:H2, :C], Symbol("1.31e-10 .* exp.(-80.0 ./ Tn)")]
Any[[:CH, :H2], [:CH2, :H], Symbol("2.9e-10 .* exp.(-1670.0 ./ Tn)")]
Any[[:CH, :H2], [:CH3], Symbol("2.92e-16 .* (Tn .^ -0.71) .* exp.(-11.6 ./ Tn)")]
Any[[:CH, :N], [:CN, :H], Symbol("2.77e-10 .* (Tn .^ -0.09)")]
Any[[:CH, :O], [:CO, :H], Symbol("6.6e-11")]
Any[[:CH, :O], [:HCOP, :E], Symbol("4.2e-13 .* exp.(-850.0 ./ Tn)")]
Any[[:CH, :O], [:OH, :C], Symbol("2.52e-11 .* exp.(-2381.0 ./ Tn)")]
Any[[:CN, :H2], [:HCN, :H], Symbol("1.8e-19 .* (Tn .^ 2.6) .* exp.(-960.0 ./ Tn)")]
Any[[:CN, :N], [:N2, :C], Symbol("9.8e-10 .* (Tn .^ -0.4)")]
Any[[:CN, :NH], [:HCN, :N], Symbol("1.7e-13 .* (Tn .^ 0.5) .* exp.(-1000.0 ./ Tn)")]
Any[[:CN, :O], [:CO, :N], Symbol("5.0e-11 .* ex

## getting minimum() to evaluate within symbolic expressions

In [61]:
T = [100, 200, 300]
println(1.0 .* T .^ 0)

T = 100
println(1.0 .* T .^ 0)

[1.0, 1.0, 1.0]
1.0


In [62]:
T = [100, 200, 300]
test = :(minimum([$:(1.0), $:(1.4e-10 .* (T .^ 0.4) .* exp.(-160 ./ T))]))
eval(test)

LoadError: MethodError: no method matching isless(::Array{Float64,1}, ::Float64)
Closest candidates are:
  isless(!Matched::Float64, ::Float64) at float.jl:465
  isless(!Matched::Missing, ::Any) at missing.jl:87
  isless(!Matched::AbstractFloat, ::AbstractFloat) at operators.jl:165
  ...

In [71]:
T = [100, 200, 300]
test = :(minimum([$:(1.0 .* T .^ 0), $:(1.4e-10 .* (T .^ 0.4) .* exp.(-160 ./ T))]))
eval(test)

3-element Array{Float64,1}:
 1.7834332719518747e-10
 5.237263783560956e-10
 8.041791591659083e-10

But what if the values which are lowest change per element? Here is a very simple example:

In [80]:
# The return should be a 5 element array with values 1, 2, 3, 3, 3.
quoted = quote
    min.([1, 2, 3, 4, 5], [3, 3, 3, 3, 3])
end

eval(quoted)

5-element Array{Int64,1}:
 1
 2
 3
 3
 3

Now to get it to work with a more complicated expression that generates an array.

In [81]:
T = [100, 121, 144, 169, 196, 225]
# sqrts: [10, 11, 12, 13, 14, 15]
# Comparing with an array of 12s, the result should be: [10, 11, 12, 12, 12, 12]

uglyexpr = quote
   min.(12. .* T .^ 0, T .^ 0.5)  
end

eval(uglyexpr)

6-element Array{Float64,1}:
 10.0
 11.0
 12.0
 12.0
 12.0
 12.0

Simplify the syntax:

In [85]:
T = [100, 121, 144, 169, 196, 225]
test = :( min.( $:(12. .* T .^ 0), $:(T .^ 0.5) ) )
eval(test)

6-element Array{Float64,1}:
 10.0
 11.0
 12.0
 12.0
 12.0
 12.0

Make sure it works when what's passed in is NOT an array:

In [86]:
T = 169
test = :( min.( $:(12. .* T .^ 0), $:(T .^ 0.5) ) )
eval(test)

12.0

In [87]:
T = 100
test = :( min.( $:(12. .* T .^ 0), $:(T .^ 0.5) ) )
eval(test)

10.0

Cool. This is what we will use, which means I need to fix the syntax I already introduced. This just requires me to replace `minimum` with `min.` and remove the square brackets in the PARAMETERS.jl file. But, it also means I need to reconverge the C, CH, HCO neutrals because they were done with an incorrect expresison.

## neutral rate coeff functions old versions



In [37]:
function make_general_k_old(A, B, C)
    #=
    returns a string for the general rate coefficient form 
    k = A * (T^B) * exp(C/T)
    checked: 4 Feb 2021
    =#
    
    k_A = string(A)
    if B != 0
        k_B = " .* (Tn .^ $(B))"
    else
        k_B = ""
    end
    if C != 0
        k_C = " .* exp.($(C) ./ Tn)"
    else
        k_C = ""
    end
    
    return k_A * k_B * k_C
end

function construct_FF_old(k0, kinf, F)
    #=
    Put together a really nasty expression
    =#
    outer_numer = "log10.($(F))"
    inner_numer = "log10.(($(k0) .* M) ./ ($(kinf))) .- 0.4 .- 0.67 .* log10.($(F))"
    inner_denom = "0.75 .- 1.27 .* log10.($(F)) .- 0.14 .* (log10.(($(k0) .* M) ./ ($(kinf))) .- 0.4 .- 0.67 .* log10.($(F)))"
    outer_denom = "1 .+ (($(inner_numer)) ./ ($(inner_denom))) .^ 2"
    FF = "10 .^ (($(outer_numer)) ./ ($(outer_denom)))"
    return FF
end


function make_Troe_old(k0_ABC, kinf_ABC, F)
    #=
    k0_ABC and kinf_ABC are lists of A, B, and C for k0 and kinf.
    =#
    k0 = make_general_k(k0_ABC...)
    kinf = make_general_k(kinf_ABC...)
    
    if F == 0
        return Symbol("($(k0) .* $(kinf) .* M) ./ ($(k0) .* M .+ $(kinf))")  # Updated to match Roger's code 4 Feb 2021
        # old formula that was wrong: return Symbol(0)
    else        
        FF = construct_FF(k0, kinf, F)
        return Symbol("$(FF) .* ($(k0) .* $(kinf)) ./ ($(k0) .* M .+ $(kinf))")  # Updated to match Roger's code 5 Feb 2021
        #old formula that was wrong: return Symbol("$(string(F)) * $(kinf) * $(k0) * M / ($(kinf) + $(k0) * M)")
    end
end

function make_modified_Troe_old(k0_ABC, kinf_ABC, kR_ABC, F)
    #=
    "Type 4" pressure dependent association reactions. 
    Roger has the result for F=0 in his code as k_tot = k2 + k0*kinf*M/(k0*M+kinf), with k2 (kR in the appendix from Vuitton)
    being the general form and using the third set of A, B, C coefficients in his file. Those columns are 0 in the file he gave me, but
    I have coded it like this though to make sure to keep the form in case I ever get one where it's not 0.
    
    k0_ABC and kinf_ABC are lists of A, B, and C for k0 and kinf.
    =#
    k0 = make_general_k(k0_ABC...)
    kinf = make_general_k(kinf_ABC...)
    kR = make_general_k(kR_ABC...)

    # old formula that was wrong: return Symbol("$(kR) + $(F) * ($(kinf) - $(kR)) * $(k0) * M / ($(kinf) - $(kR) + $(k0) * M)")
    if F == 0
        return Symbol("$(kR) .+ ($(k0) .* $(kinf) .* M) ./ ($(k0) .* M .+ $(kinf))")
    else
        FF = construct_FF(k0, kinf, F)
        
        # for this one, we need the minimum of the following two expressions. Because of the weird way I have to do this, I will add
        # :(minimum([$:(A), $:(B)])) around the two returned items A and B once it returns its stuff.
        return Symbol("$(kinf)"), Symbol("$(kR) .+ ($(FF) .* $(k0) .* $(kinf) .* M) ./ ($(k0) .* M .+ $(kinf))")
    end
end


make_modified_Troe_old (generic function with 1 method)